In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import numpy as np



In [1]:
path_poscon_openphenom = "/home/maxime/synrepos/phenoseeker/tmp/openphenom/48_plates_2_combi/results/maps_jcp2022.csv"
path_plates_openphenom = "/home/maxime/synrepos/phenoseeker/tmp/openphenom/48_plates_2_combi/results/maps_plate.csv"
path_poscon_dinov2_g = "/home/maxime/synrepos/phenoseeker/tmp/dinov2_g/48_plates_2_combi/results/maps_jcp2022.csv"
path_plates_dinov2_g = "/home/maxime/synrepos/phenoseeker/tmp/dinov2_g/48_plates_2_combi/results/maps_plate.csv"
path_poscon_dinov2_s = "/home/maxime/synrepos/phenoseeker/tmp/dinov2_s/48_plates_2_combi/results/maps_jcp2022.csv"
path_plates_dinov2_s = "/home/maxime/synrepos/phenoseeker/tmp/dinov2_s/48_plates_2_combi/results/maps_plate.csv"
path_poscon_resnet50_mean = "/home/maxime/synrepos/phenoseeker/tmp/resnet50_mean/48_plates_2_combi/results/maps_jcp2022.csv"
path_plates_resnet50_mean = "/home/maxime/synrepos/phenoseeker/tmp/resnet50_mean/48_plates_2_combi/results/maps_plate.csv"
path_poscon_resnet50_median = "/home/maxime/synrepos/phenoseeker/tmp/resnet50_median/48_plates_2_combi/results/maps_jcp2022.csv"
path_plates_resnet50_median = "/home/maxime/synrepos/phenoseeker/tmp/resnet50_median/48_plates_2_combi/results/maps_plate.csv"
path_poscon_chada = "/home/maxime/synrepos/phenoseeker/tmp/chada/48_plates_2_combi/results/maps_jcp2022.csv"
path_plates_chada = "/home/maxime/synrepos/phenoseeker/tmp/chada/48_plates_2_combi/results/maps_plate.csv"

In [1]:
path_poscon_openphenom = "/projects/synsight/repos/phenoseeker/tmp/openphenom/results/maps_jcp2022.csv"
path_plates_openphenom = "/projects/synsight/repos/phenoseeker/tmp/openphenom/results/maps_plate.csv"
path_poscon_dinov2_g = "/projects/synsight/repos/phenoseeker/tmp/dinov2_g/48_plates_2_combi/results/maps_jcp2022.csv"
path_plates_dinov2_g = "/projects/synsight/repos/phenoseeker/tmp/dinov2_g/48_plates_2_combi/results/maps_plate.csv"
path_poscon_chada = "/projects/synsight/repos/phenoseeker/tmp/chada/48_plates_2_combi/results/maps_jcp2022.csv"
path_plates_chada = "/projects/synsight/repos/phenoseeker/tmp/chada/48_plates_2_combi/results/maps_plate.csv"

In [28]:
#df_poscon = pd.read_csv('/projects/synsight/repos/phenospace/normalisation/experiments/dinov2/test_all_transform_9_plates/results/maps_df_poscon.csv')
#df_plates = pd.read_csv('/projects/synsight/repos/phenospace/normalisation/experiments/dinov2/test_all_transform_9_plates/results/maps_df_controls_Plate.csv')


# Poscon mAP

In [3]:
def preprocess(path):
    df_raw = pd.read_csv(path)
    df_raw.columns = [col.replace('mAP', '').replace('raw_', '').replace('(', '').replace(')', '').replace('raw', 'Raw').strip() for col in df_raw.columns]
    df = df_raw.drop(columns=['Number of Queries'])
    df = df.set_index('Label').drop(index='Mean mAP').astype(float)
    return df.mean()


In [53]:
def plot_curve_comparison(encoders_dict):
    """
    Plot mAP values for multiple encoders with custom styling.
    Highlights specific points for each encoder (No Normalisation, Best Normalisation) with unified colors.

    Args:
        encoders_dict: Dictionary where keys are encoder names and values are lists containing:
                       [poscon_values, batch_effect_values, best_normalisation_index].
    """
    plt.figure(figsize=(14, 8))

    # Define marker styles for each encoder
    marker_styles = ['o', 's', 'D', '^', 'v', 'P', 'X']  # Extend as needed
    unified_no_norm_color = '#D55E00'  # Unified color for "No Normalisation"
    unified_best_norm_color = '#009E73'  # Unified color for "Best Normalisation"
    unified_random_color = '#F0E442'  # Unified color for "Random Values"

    shape_patches = []  # For shape legend

    for i, (encoder, values) in enumerate(encoders_dict.items()):
        mean_values_1, mean_values_2, best_normalisation_index = values

        # Assign unique marker
        marker = marker_styles[i % len(marker_styles)]

        # General scatter plot with lighter points
        plt.scatter(mean_values_2, mean_values_1, 
                    color="grey", alpha=1, s=20, marker=marker)

        # Highlight the first point (No Normalisation)
        plt.scatter(mean_values_2['Embeddings_Raw'], mean_values_1['Embeddings_Raw'], 
                    color=unified_no_norm_color, alpha=1.0, s=100, edgecolor='black', linewidth=1, 
                    marker=marker)
        # Highlight the best normalisation point
        plt.scatter(mean_values_2["Embeddings Random"], mean_values_1["Embeddings Random"], 
                    color=unified_random_color, alpha=1.0, s=100, edgecolor='black', linewidth=1, 
                    marker=marker)
        # Highlight the best normalisation point
        plt.scatter(mean_values_2[best_normalisation_index], mean_values_1[best_normalisation_index], 
                    color=unified_best_norm_color, alpha=1.0, s=100, edgecolor='black', linewidth=1, 
                    marker=marker)

        # Add to shape legend if not already added
        if marker not in [line.get_marker() for line in shape_patches]:
            shape_patches.append(mlines.Line2D([], [], color='black', marker=marker, linestyle='None', markersize=10, label=f'{encoder}'))


    # Add labels
    plt.xlabel('mAP - Positive Control Molecules Retrieval', fontsize=14, labelpad=10)
    plt.ylabel('mAP - Plates Retrieval', fontsize=14, labelpad=10)

    # Refine grid lines
    plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.6)

    # Customize ticks
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # Add legends
    color_legend = [
        mpatches.Patch(color=unified_no_norm_color, label='No Normalisation'),
        mpatches.Patch(color=unified_best_norm_color, label='Best Normalisation')
    ]

    plt.legend(handles=shape_patches, loc='upper left', bbox_to_anchor=(1.05, 0.5), fontsize=12, title='Shape Legend')

    # Tight layout for better use of space and high resolution
    plt.tight_layout()
    plt.show()


In [26]:
paths = {
    "poscon_openphenom": "/home/maxime/synrepos/phenoseeker/tmp/openphenom/48_plates_2_combi/results/maps_jcp2022.csv",
    "plates_openphenom": "/home/maxime/synrepos/phenoseeker/tmp/openphenom/48_plates_2_combi/results/maps_plate.csv",
    "poscon_dinov2_g": "/home/maxime/synrepos/phenoseeker/tmp/dinov2_g/48_plates_2_combi/results/maps_jcp2022.csv",
    "plates_dinov2_g": "/home/maxime/synrepos/phenoseeker/tmp/dinov2_g/48_plates_2_combi/results/maps_plate.csv",
    "poscon_dinov2_s": "/home/maxime/synrepos/phenoseeker/tmp/dinov2_s/48_plates_2_combi/results/maps_jcp2022.csv",
    "plates_dinov2_s": "/home/maxime/synrepos/phenoseeker/tmp/dinov2_s/48_plates_2_combi/results/maps_plate.csv",
    "poscon_resnet50_mean": "/home/maxime/synrepos/phenoseeker/tmp/resnet50_mean/48_plates_2_combi/results/maps_jcp2022.csv",
    "plates_resnet50_mean": "/home/maxime/synrepos/phenoseeker/tmp/resnet50_mean/48_plates_2_combi/results/maps_plate.csv",
    "poscon_resnet50_median": "/home/maxime/synrepos/phenoseeker/tmp/resnet50_median/48_plates_2_combi/results/maps_jcp2022.csv",
    "plates_resnet50_median": "/home/maxime/synrepos/phenoseeker/tmp/resnet50_median/48_plates_2_combi/results/maps_plate.csv",
    "poscon_chada": "/home/maxime/synrepos/phenoseeker/tmp/chada/48_plates_2_combi/results/maps_jcp2022.csv",
    "plates_chada": "/home/maxime/synrepos/phenoseeker/tmp/chada/48_plates_2_combi/results/maps_plate.csv"
}



# Valeurs d'"Embeddings Random" associées
embeddings_random_values = {
    "poscon_dinov2_g": 0.125,
    "plates_dinov2_g": 0.02,
    "poscon_dinov2_s": 0.125,
    "plates_dinov2_s": 0.02,
    "poscon_resnet50_mean": 0.125,
    "plates_resnet50_mean": 0.02,
    "poscon_resnet50_median": 0.125,
    "plates_resnet50_median": 0.02,
    "poscon_chada": 0.125,
    "plates_chada": 0.02,
    "poscon_openphenom": 0.125,
    "plates_openphenom": 0.02,
}

# Traitement automatisé
processed_data = {}

for key, path in paths.items():
    df = preprocess(path)
    if key in embeddings_random_values:
        df['Embeddings Random'] = embeddings_random_values[key]
    processed_data[key] = df



In [ ]:
processed_data['poscon_resnet50_median']

In [ ]:
encoder = "chada"
df = pd.concat([processed_data[f'plates_{encoder}'], processed_data[f'poscon_{encoder}']], axis=1)
df.columns = ['Batch_effect', 'Poscon']
df.sort_values(by="Poscon",ascending=False).head(15)

In [45]:
all_encoder = {'Dinov2_g': [ processed_data["plates_dinov2_g"],processed_data["poscon_dinov2_g"],  'Embeddings_Raw__ZCA_N_C__rZMi'], 
               'Open_Phenom': [ processed_data["plates_openphenom"],processed_data["poscon_openphenom"], 'Embeddings_Raw__rZMi'], 
               'Dinov2_s': [ processed_data["plates_dinov2_s"],processed_data["poscon_dinov2_s"], 'Embeddings_Raw__ZCA-cor_N_C__rZmi'], 
               'Resnet50_mean': [ processed_data["plates_resnet50_mean"],processed_data["poscon_resnet50_mean"], 'Embeddings_Raw__ZCA-cor_C__rZMs'], 
               'Resnet50_median': [ processed_data["plates_resnet50_median"],processed_data["poscon_resnet50_median"], 'Embeddings_Raw__rZMs'], 
               'ChAda': [ processed_data["plates_chada"],processed_data["poscon_chada"], 'Embeddings_Raw__ZCA_N_C__rZMs']}

In [ ]:
plot_curve_comparison(all_encoder)

In [1]:
import pandas as pd

In [ ]:
pd.read_json('/projects/synsight/repos/phenoseeker/scripts/balanced_plates.json')